# ODI Complaints EDA

This notebook will be for our exploration and first cleaning steps of the complaint dataset. Other notebooks will be used to keep things like different models separate.

The `src/` folder will be where the finalized workflow for each step is done so a main file can run them all in order and keep everything clean and reproducible. The files there won't be setup for exploring and getting outputs.

## Quick Setup

1. Run cells from top to bottom, first few cells setup the pathing
2. The notebook loads the combined processed parquet by default
3. A DataFrame named `df` is created for the combined with options for the separate datasets

If the file is missing, run the pipeline first:
- Windows: `./scripts/run_pipeline_windows.ps1`
- macOS/Linux: `./scripts/run_pipeline_mac_linux.sh`

In [1]:
# Imports
from pathlib import Path

import pandas as pd
from IPython.display import display

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 220)
pd.set_option("display.max_colwidth", 120)

Load the combined processed dataset

In [2]:
# This notebook assumes the project setup matches the repo defaults

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "data" / "processed").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
COMBINED_PATH = PROCESSED_DIR / "odi_complaints_combined.parquet"

if not COMBINED_PATH.exists():
    raise FileNotFoundError(
        f"Could not find {COMBINED_PATH}. Run the pipeline first to create processed outputs."
    )

df = pd.read_parquet(COMBINED_PATH)
df = df.drop(columns=["source_zip", "source_file"], errors="ignore")

# Uncomment the following lines to load the individual year datasets if you want to compare them separately.
# OLD_PATH = PROCESSED_DIR / "COMPLAINTS_RECEIVED_2020-2024_processed.parquet"
# NEW_PATH = PROCESSED_DIR / "COMPLAINTS_RECEIVED_2025-2026_processed.parquet"
# df_2020_2024 = pd.read_parquet(OLD_PATH)
# df_2025_2026 = pd.read_parquet(NEW_PATH)

print("Loaded:", COMBINED_PATH.name)
print("Shape:", df.shape)

Loaded: odi_complaints_combined.parquet
Shape: (545231, 49)


## Initial Exploration

In [3]:
print("Column names (first 25)")
display(pd.Series(df.columns, name="column").head(25).to_frame())

print("Data types (first 25)")
display(df.dtypes.rename("dtype").reset_index(name="column").head(25))

print("First 5 rows")
display(df.head(5))

Column names (first 25)


,column
0,cmplid
1,odino
2,mfr_name
3,maketxt
4,modeltxt
5,yeartxt
6,crash
7,faildate
8,fire
9,injured


Data types (first 25)


,index,column
0,cmplid,string
1,odino,string
2,mfr_name,string
3,maketxt,string
4,modeltxt,string
5,yeartxt,string
6,crash,string
7,faildate,datetime64[us]
8,fire,string
9,injured,string


First 5 rows


,cmplid,odino,mfr_name,maketxt,modeltxt,yeartxt,crash,faildate,fire,injured,deaths,compdesc,city,state,vin,datea,ldate,miles,occurences,cdescr,cmpl_type,police_rpt_yn,purch_dt,orig_owner_yn,anti_brakes_yn,cruise_cont_yn,num_cyls,drive_train,fuel_sys,fuel_type,trans_type,veh_speed,dot,tire_size,loc_of_tire,tire_fail_type,orig_equip_yn,manuf_dt,seat_type,restraint_type,dealer_name,dealer_tel,dealer_city,dealer_state,dealer_zip,prod_type,repaired_yn,medical_attn,vehicles_towed_yn
0,1633421,11292384,Honda (American Honda Motor Co.),HONDA,ACCORD,2018,N,2019-12-21,N,0,0,SERVICE BRAKES,PHILADELPHIA,PA,1HGCV2F38JA,2020-01-01,2020-01-01,4,<NA>,"DRIVING AT THE HIGHWAY, CAR SUDDENLY SLOW DOWN FROM 70MPH TO 40-50MPH,THERE WERE ANY CAR IN FRONT OF ME! I HAVE NOTH...",IVOQ,N,<NA>,N,N,N,<NA>,<NA>,<NA>,<NA>,<NA>,68,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,V,<NA>,N,N
1,1633422,11292384,Honda (American Honda Motor Co.),HONDA,ACCORD,2018,N,2019-12-21,N,0,0,ELECTRICAL SYSTEM,PHILADELPHIA,PA,1HGCV2F38JA,2020-01-01,2020-01-01,4,<NA>,"DRIVING AT THE HIGHWAY, CAR SUDDENLY SLOW DOWN FROM 70MPH TO 40-50MPH,THERE WERE ANY CAR IN FRONT OF ME! I HAVE NOTH...",IVOQ,N,<NA>,N,N,N,<NA>,<NA>,<NA>,<NA>,<NA>,68,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,V,<NA>,N,N
2,1633423,11292384,Honda (American Honda Motor Co.),HONDA,ACCORD,2018,N,2019-12-21,N,0,0,ENGINE,PHILADELPHIA,PA,1HGCV2F38JA,2020-01-01,2020-01-01,4,<NA>,"DRIVING AT THE HIGHWAY, CAR SUDDENLY SLOW DOWN FROM 70MPH TO 40-50MPH,THERE WERE ANY CAR IN FRONT OF ME! I HAVE NOTH...",IVOQ,N,<NA>,N,N,N,<NA>,<NA>,<NA>,<NA>,<NA>,68,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,V,<NA>,N,N
3,1633424,11292385,Ford Motor Company,FORD,EXPLORER,2020,N,2019-12-26,N,0,0,ELECTRICAL SYSTEM,MEHERRIN,VA,1FM5K8GC8LG,2020-01-01,2020-01-01,5300,<NA>,DEEP SLEEP MODE ACTIVATES AFTER 2 DAYS. MOST RECENT EXPERIENCE INVOLVED HAVING TO JUMP START THE CAR AFTER SETTING ...,IVOQ,N,<NA>,N,N,N,<NA>,<NA>,<NA>,<NA>,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,V,<NA>,N,N
4,1633425,11292386,"General Motors, LLC",CHEVROLET,VOLT,2017,N,2019-07-12,N,0,0,SERVICE BRAKES,SAN ANTONIO,TX,1G1RB6S52HU,2020-01-01,2020-01-01,15000,<NA>,"WHILE DRIVING ON CITY STREETS AND HIGHWAYS, THE ADAPTIVE CRUISE CONTROL WILL NOT ENGAGE 25% OF THE TIME AND WILL DIS...",IVOQ,N,<NA>,N,N,N,<NA>,<NA>,<NA>,<NA>,<NA>,70,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,V,<NA>,N,N


Get a quick count and percentage of null values by column

In [4]:
# Null summary (overall)
null_summary = (
    df.isna()
      .sum()
      .sort_values(ascending=False)
      .rename("null_count")
      .to_frame()
)
null_summary["null_pct"] = (null_summary["null_count"] / len(df) * 100).round(2)

display(null_summary)

,null_count,null_pct
fuel_sys,544940,99.95
tire_size,544882,99.94
tire_fail_type,544483,99.86
manuf_dt,544323,99.83
restraint_type,544283,99.83
seat_type,544188,99.81
trans_type,543621,99.70
orig_equip_yn,543089,99.61
dot,542505,99.50
purch_dt,542416,99.48


There are a lot of null values, but some of it is because many columns are fields for a specific product type. So if 25% of the products are Type A and Type A products have five fields specific to it, then they'd have a minimum null percentage of 75%. This checks the null percentages based on the product type.

In [12]:
prod_type_counts = (
    df["prod_type"].value_counts(dropna=False)
    .rename_axis("prod_type")
    .reset_index(name="row_count")
)
prod_type_counts["row_pct"] = (prod_type_counts["row_count"] / len(df) * 100).round(2)

print("Rows by product type")
display(prod_type_counts)

missingness_by_prod = pd.DataFrame(
    {
        "overall_null_pct": (df.isna().mean() * 100).round(2)
    }
)

for prod_value in prod_type_counts["prod_type"].tolist():
    mask = df["prod_type"] == prod_value
    safe_label = str(prod_value).lower().replace(" ", "_")
    missingness_by_prod[f"null_pct_{safe_label}"] = (df.loc[mask].isna().mean() * 100).round(2)
    missingness_by_prod[f"non_null_pct_{safe_label}"] = (df.loc[mask].notna().mean() * 100).round(2)

missingness_by_prod = missingness_by_prod.sort_values("overall_null_pct", ascending=False)

print("Missingness by product type (all columns)")
display(missingness_by_prod)

print("Likely structural sparsity (very high overall nulls, but usable within at least one product type)")
structural_sparse_candidates = missingness_by_prod[
    (missingness_by_prod["overall_null_pct"] >= 80)
    & (missingness_by_prod.filter(regex=r"^non_null_pct_").max(axis=1) >= 20)
]
display(structural_sparse_candidates)


Rows by product type


,prod_type,row_count,row_pct
0,V,537936,98.66
1,T,3820,0.7
2,E,2142,0.39
3,C,1326,0.24
4,<NA>,7,0.0


Missingness by product type (all columns)


,overall_null_pct,null_pct_v,non_null_pct_v,null_pct_t,non_null_pct_t,null_pct_e,non_null_pct_e,null_pct_c,non_null_pct_c,null_pct_<na>,non_null_pct_<na>
fuel_sys,99.95,99.95,0.05,100.00,0.00,100.00,0.00,100.00,0.00,NaN,NaN
tire_size,99.94,100.00,0.00,90.86,9.14,100.00,0.00,100.00,0.00,NaN,NaN
tire_fail_type,99.86,100.00,0.00,80.42,19.58,100.00,0.00,100.00,0.00,NaN,NaN
restraint_type,99.83,100.00,0.00,100.00,0.00,100.00,0.00,28.51,71.49,NaN,NaN
manuf_dt,99.83,100.00,0.00,100.00,0.00,100.00,0.00,31.52,68.48,NaN,NaN
seat_type,99.81,100.00,0.00,100.00,0.00,100.00,0.00,21.34,78.66,NaN,NaN
trans_type,99.70,99.70,0.30,100.00,0.00,100.00,0.00,100.00,0.00,NaN,NaN
orig_equip_yn,99.61,100.00,0.00,100.00,0.00,0.00,100.00,100.00,0.00,NaN,NaN
dot,99.50,100.00,0.00,28.64,71.36,100.00,0.00,100.00,0.00,NaN,NaN
purch_dt,99.48,99.70,0.30,100.00,0.00,44.63,55.37,97.81,2.19,NaN,NaN


Likely structural sparsity (very high overall nulls, but usable within at least one product type)


,overall_null_pct,null_pct_v,non_null_pct_v,null_pct_t,non_null_pct_t,null_pct_e,non_null_pct_e,null_pct_c,non_null_pct_c,null_pct_<na>,non_null_pct_<na>
restraint_type,99.83,100.0,0.0,100.00,0.00,100.00,0.00,28.51,71.49,NaN,NaN
manuf_dt,99.83,100.0,0.0,100.00,0.00,100.00,0.00,31.52,68.48,NaN,NaN
seat_type,99.81,100.0,0.0,100.00,0.00,100.00,0.00,21.34,78.66,NaN,NaN
orig_equip_yn,99.61,100.0,0.0,100.00,0.00,0.00,100.00,100.00,0.00,NaN,NaN
dot,99.50,100.0,0.0,28.64,71.36,100.00,0.00,100.00,0.00,NaN,NaN
purch_dt,99.48,99.7,0.3,100.00,0.00,44.63,55.37,97.81,2.19,NaN,NaN
loc_of_tire,99.42,100.0,0.0,17.91,82.09,100.00,0.00,100.00,0.00,NaN,NaN
repaired_yn,99.30,100.0,0.0,0.00,100.00,100.00,0.00,100.00,0.00,NaN,NaN


## Scratchpad 

Examples:
- `df.columns.tolist()`
- `df[["maketxt", "modeltxt", "yeartxt", "compdesc"]].head(20)`
- `df["compdesc"].value_counts(dropna=False).head(20)`
- `df.groupby("maketxt").size().sort_values(ascending=False).head(20)`
- `df[df["crash"] == "Y"].shape`
- `df[df["prod_type"] == "V"].isna().mean().sort_values(ascending=False).head(20)`


In [ ]:
# Vehicle-only view (likely the main analysis cohort)
df_vehicle = df[df["prod_type"] == "V"].copy()

print("Vehicle-only rows:", len(df_vehicle))

vehicle_null_summary = (
    df_vehicle.isna()
    .sum()
    .sort_values(ascending=False)
    .rename("null_count")
    .to_frame()
)
vehicle_null_summary["null_pct"] = (vehicle_null_summary["null_count"] / len(df_vehicle) * 100).round(2)

display(vehicle_null_summary.head(20))


# display(df_vehicle[["maketxt", "modeltxt", "compdesc"]].head(20))
# display(df_vehicle.groupby("compdesc").size().sort_values(ascending=False).head(30))
